In [10]:
import pyodbc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [11]:
DRIVER = '{ODBC Driver 17 for SQL Server}'
SERVER = 'LAPTOP-BSJ8KIE2'
DATABASE = 'voka_db'


CONNECTION_STRING = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'

connection = pyodbc.connect(CONNECTION_STRING)

query = """
SELECT 
c.crm_campagne_naam,c.crm_campagne_naam_in_email,c.crm_campagne_type_campagne,c.crm_campagne_soort_campagne,
s.crm_sessie_activiteitstype,s.crm_sessie_product,s.crm_sessie_thema_naam,i.crm_inschrijving_aanwezig_afwezig,i.crm_inschrijving_bron,i.crm_inschrijving_facturatie_bedrag,
co.crm_contact_functietitel,co.crm_contact_status,co.crm_contact_voka_medewerker,
p.crm_persoon_mail_regio_antwerpen_waasland,p.crm_persoon_mail_regio_brussel_hoofdstedelijk_gewest,p.crm_persoon_mail_regio_limburg,p.crm_persoon_mail_regio_mechelen_kempen,
p.crm_persoon_mail_regio_oost_vlaanderen,p.crm_persoon_mail_regio_vlaams_brabant,p.crm_persoon_mail_regio_voka_nationaal,p.crm_persoon_mail_regio_west_vlaanderen,p.crm_persoon_mail_thema_duurzaamheid,
p.crm_persoon_mail_thema_financieel_fiscaal,p.crm_persoon_mail_thema_innovatie,p.crm_persoon_mail_thema_internationaal_ondernemen,p.crm_persoon_mail_thema_mobiliteit,p.crm_persoon_mail_thema_omgeving,
p.crm_persoon_mail_thema_sales_marketing_communicatie,p.crm_persoon_mail_thema_strategie_en_algemeen_management,p.crm_Persoon_Mail_thema_talent,p.crm_persoon_mail_thema_welzijn,p.crm_persoon_mail_type_bevraging,
p.crm_persoon_mail_type_communities_en_projecten,p.crm_persoon_mail_type_netwerkevenementen,p.crm_persoon_mail_type_nieuwsbrieven,p.crm_persoon_mail_type_opleidingen,p.crm_persoon_mail_type_persberichten_belangrijke_meldingen,
p.crm_persoon_marketingcommunicatie,p.crm_persoon_persoonsnr,p.crm_persoon_reden_van_status,a.crm_account_adres_geografische_regio,a.crm_account_adres_geografische_subregio,a.crm_account_adres_land,a.crm_account_adres_plaats,a.crm_account_adres_postcode,a.crm_account_adres_provincie,a.crm_account_industriezone_naam,a.crm_account_is_voka_entiteit,a.crm_account_ondernemingsaard,a.crm_account_ondernemingstype,a.crm_account_primaire_activiteit,a.crm_account_reden_van_status,a.crm_account_status,a.crm_account_voka_nr,
fd.crm_financieledata_aantal_maanden,fd.crm_financieledata_boekjaar,fd.crm_financieledata_fte,fd.crm_financieledata_gewijzigd_op,fd.crm_financieledata_toegevoegde_waarde,
ik.crm_info_en_klachten_aanvraag,ik.crm_info_en_klachten_status,
act.crm_activiteitscode_naam, act.crm_activiteitscode_status
FROM
    campagne as c
INNER JOIN sessie AS s ON s.crm_campagne_id = c.crm_campagne_id
INNER JOIN sessie_inschrijving AS si ON si.crm_sessie_id = s.crm_sessie_id
INNER JOIN inschrijving AS i ON si.crm_inschrijving_id = i.crm_inschrijving_id
INNER JOIN contact AS co ON i.crm_contact_id = co.crm_contact_id
INNER JOIN persoon AS p ON co.crm_persoon_id = p.crm_persoon_id
INNER JOIN account AS a ON a.crm_account_id = co.crm_contact_account
INNER JOIN financiele_data AS fd ON a.crm_account_id= fd.crm_account_id
INNER JOIN info_en_klachten AS ik ON ik.crm_account_id= a.crm_account_id
INNER JOIN account_activiteitscode AS aa ON a.crm_account_id= aa.crm_account_id
INNER JOIN activiteitscode AS act ON act.crm_activiteitscode_id = aa.crm_activiteitscode_id
"""

querydf = pd.read_sql(query, connection)

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_9868\1528251108.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  querydf = pd.read_sql(query, connection)


In [12]:
querydf.head()

,crm_campagne_naam,crm_campagne_naam_in_email,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_sessie_activiteitstype,crm_sessie_product,crm_sessie_thema_naam,crm_inschrijving_aanwezig_afwezig,crm_inschrijving_bron,crm_inschrijving_facturatie_bedrag,...,crm_account_voka_nr,crm_financieledata_aantal_maanden,crm_financieledata_boekjaar,crm_financieledata_fte,crm_financieledata_gewijzigd_op,crm_financieledata_toegevoegde_waarde,crm_info_en_klachten_aanvraag,crm_info_en_klachten_status,crm_activiteitscode_naam,crm_activiteitscode_status
0,OV-NW-Voka Vista - Community Event-Grootmeeste...,Community Event: Grootmeesters van de groei,Netwerkevenement,Offline,NW Voka Vista,OV-Netwerkactiviteit-provinciaal,Groeien,Aanwezig,Website,60.0,...,82334,NaN,2005,0.1,2018-10-15 05:25:39,NaN,450B6751-09C4-ED11-83FF-6045BD895805,Gesloten,Telecom & IT,Actief
1,OV-NW-Voka Vista - Community Event-Grootmeeste...,Community Event: Grootmeesters van de groei,Netwerkevenement,Offline,NW Voka Vista,OV-Netwerkactiviteit-provinciaal,Groeien,Aanwezig,Website,60.0,...,82334,NaN,2005,0.1,2018-10-15 05:25:39,NaN,5A0A80F0-5FA1-ED11-AAD0-6045BD895BFB,Gesloten,Telecom & IT,Actief
2,OV-NW-Voka Vista - Community Event-Grootmeeste...,Community Event: Grootmeesters van de groei,Netwerkevenement,Offline,NW Voka Vista,OV-Netwerkactiviteit-provinciaal,Groeien,Aanwezig,Website,60.0,...,82334,NaN,2005,0.1,2018-10-15 05:25:39,NaN,88735758-F453-EB11-8117-001DD8B72B61,Gesloten,Telecom & IT,Actief
3,OV-NW-Voka Vista - Community Event-Grootmeeste...,Community Event: Grootmeesters van de groei,Netwerkevenement,Offline,NW Voka Vista,OV-Netwerkactiviteit-provinciaal,Groeien,Aanwezig,Website,60.0,...,82334,NaN,2005,0.1,2018-10-15 05:25:39,NaN,941D45D3-8C56-ED11-BBA2-6045BD8956C9,Gesloten,Telecom & IT,Actief
4,OV-NW-Voka Vista - Community Event-Grootmeeste...,Community Event: Grootmeesters van de groei,Netwerkevenement,Offline,NW Voka Vista,OV-Netwerkactiviteit-provinciaal,Groeien,Aanwezig,Website,60.0,...,82334,NaN,2005,0.1,2018-10-15 05:25:39,NaN,2350EA3D-B281-EB11-811D-001DD8B72B62,Gesloten,Telecom & IT,Actief


In [13]:
querydf.value_counts

<bound method DataFrame.value_counts of                                         crm_campagne_naam  \
0       OV-NW-Voka Vista - Community Event-Grootmeeste...   
1       OV-NW-Voka Vista - Community Event-Grootmeeste...   
2       OV-NW-Voka Vista - Community Event-Grootmeeste...   
3       OV-NW-Voka Vista - Community Event-Grootmeeste...   
4       OV-NW-Voka Vista - Community Event-Grootmeeste...   
...                                                   ...   
208592  OV-NW- Quick Refresh - webinar: Arbeidsdeal me...   
208593  OV-NW- Quick Refresh - webinar: Arbeidsdeal me...   
208594  OV-NW- Quick Refresh - webinar: Arbeidsdeal me...   
208595  OV-NW- Quick Refresh - webinar: Arbeidsdeal me...   
208596  OV-NW- Quick Refresh - webinar: Arbeidsdeal me...   

                               crm_campagne_naam_in_email  \
0             Community Event: Grootmeesters van de groei   
1             Community Event: Grootmeesters van de groei   
2             Community Event: Grootmeesters

In [14]:
data = querydf.dropna()
data = data.drop_duplicates()

In [15]:
data.value_counts

<bound method DataFrame.value_counts of                                         crm_campagne_naam  \
568     OV-NW-20210913-VokaUpdate-BigRefresh 11-Najaar...   
569     OV-NW-20210913-VokaUpdate-BigRefresh 11-Najaar...   
578     OV-NW-20210913-VokaUpdate-BigRefresh 11-Najaar...   
579     OV-NW-20210913-VokaUpdate-BigRefresh 11-Najaar...   
580     OV-NW-20210913-VokaUpdate-BigRefresh 11-Najaar...   
...                                                   ...   
208562  OV-NW-211122-VokaUpdate-QuickRefresh 2-Najaar2021   
208563  OV-NW-211122-VokaUpdate-QuickRefresh 2-Najaar2021   
208564  OV-NW-211122-VokaUpdate-QuickRefresh 2-Najaar2021   
208565  OV-NW-211122-VokaUpdate-QuickRefresh 2-Najaar2021   
208566  OV-NW-211122-VokaUpdate-QuickRefresh 2-Najaar2021   

                               crm_campagne_naam_in_email  \
568     Voka Big Refresh - de grote najaarsupdate op m...   
569     Voka Big Refresh - de grote najaarsupdate op m...   
578     Voka Big Refresh - de grote najaarsu

In [16]:
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


df = pd.get_dummies(data, drop_first=True)


# Define the feature columns and target variable
X = df.drop(columns=['crm_campagne_type_campagne',
    'crm_campagne_soort_campagne','crm_sessie_activiteitstype',
    'crm_sessie_product',
    'crm_sessie_thema_naam'])  # Features
y = df[[
    'crm_campagne_soort_campagne']]  # Target variable


# Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns
# Load your dataset (replace 'data.csv' with your data file)
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Identify datetime columns
datetime_columns = X.select_dtypes(include=['datetime64']).columns

# Create a preprocessor that scales numeric data, one-hot encodes categorical data, and excludes datetime data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), make_column_selector(dtype_exclude=["datetime64"])(X)),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

# Create a pipeline that preprocesses data and trains the logistic regression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')
#roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
#print(f"ROC AUC: {roc_auc:.2f}")

KeyError: "['crm_campagne_type_campagne', 'crm_campagne_soort_campagne', 'crm_sessie_activiteitstype', 'crm_sessie_product', 'crm_sessie_thema_naam'] not found in axis"